## Cooperative-Competitive Maximum Coverage Location Problem  solved by Gurobi

In [ ]:
import time

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import math


def solve_hulatang_location_model(demand_points, candidate_sites, existing_stores, p, parameters, distance_matrix, allowed_pairs=None):
    """
    Solve the cooperative–competitive Hulatang location model with Gurobi (supports service-radius constraints).

    Args:
        demand_points (dict): demand dictionary {demand_id: market_value_V}
        candidate_sites (dict): candidate dictionary {candidate_id: {'coords': (x, y), 'rent': rent_F, 'norm_coords': (nx, ny)}}
        existing_stores (dict): existing store dictionary {store_id: {'coords': (x, y), 'Ak': score_A, 'Pk': score_P, 'norm_coords': (nx, ny)}}
        p (int): number of new stores to open
        parameters (dict): model params {'w1', 'w2', 'w3', 'w4', 'delta_agg', 'delta_comp'}
        distance_matrix (dict): candidate-to-existing distance matrix {(candidate_id, store_id): distance_norm}
        allowed_pairs (set[tuple]|list[tuple]|None): allowed (i, j) service pairs (generated by radius using normalized coords).

    Returns:
        tuple: (selected_sites, objective_value, covered_demand, total_demand, coverage_rate)
    """



    # --- 1. Preprocessing: compute agglomeration benefit (B_j) and competition cost (C_j) for each candidate ---
    B, C = {}, {}
    delta_agg_sq = parameters['delta_agg'] ** 2
    delta_comp_sq = parameters['delta_comp'] ** 2

    for j in candidate_sites.keys():
        total_agglomeration_benefit = 0
        total_competition_cost = 0
        for k, store_data in existing_stores.items():
            dist_sq = distance_matrix[j, k] ** 2
            total_agglomeration_benefit += store_data['Ak'] * math.exp(-dist_sq / delta_agg_sq)
            total_competition_cost += store_data['Pk'] * math.exp(-dist_sq / delta_comp_sq)
        B[j] = total_agglomeration_benefit
        C[j] = total_competition_cost

    # --- 2. Gurobi model construction ---
    model = gp.Model("Hulatang_Location_Optimization")
    model.setParam('OutputFlag', False) # silence solver log


    I = list(demand_points.keys())  # demand index set
    J = list(candidate_sites.keys())  # candidate index set

    # Decision variable X
    X = model.addVars(J, vtype=GRB.BINARY, name="X")

    # Decision variable Y: if allowed_pairs provided, create only for those (i, j)
    if allowed_pairs is not None:
        allowed_pairs = list(allowed_pairs)
        Y = model.addVars(allowed_pairs, vtype=GRB.BINARY, name="Y")
        # Precompute index for later aggregation
        from collections import defaultdict
        J_by_i = defaultdict(list)
        for (i, j) in allowed_pairs:
            J_by_i[i].append(j)
    else:
        Y = model.addVars(I, J, vtype=GRB.BINARY, name="Y")
        J_by_i = None

    # Objective
    if allowed_pairs is not None:
        coverage_term = gp.quicksum(demand_points[i] * Y[i, j] for (i, j) in allowed_pairs)
    else:
        coverage_term = gp.quicksum(demand_points[i] * Y[i, j] for i in I for j in J)

    objective = (
        parameters['w1'] * coverage_term +
        parameters['w3'] * gp.quicksum(B[j] * X[j] for j in J) -
        parameters['w4'] * gp.quicksum(C[j] * X[j] for j in J) -
        parameters['w2'] * gp.quicksum(candidate_sites[j]['rent'] * X[j] for j in J)
    )
    model.setObjective(objective, GRB.MAXIMIZE)

    # Constraint: number of opened stores
    model.addConstr(gp.quicksum(X[j] for j in J) == p, name="Total_Stores_Constraint")

    # Constraint: each demand i can be served by at most one j
    if allowed_pairs is not None:
        for i in I:
            js = J_by_i.get(i, [])
            if len(js) > 0:
                model.addConstr(gp.quicksum(Y[i, j] for j in js) <= 1, name=f"Demand_Service_Constraint_{i}")
            else:
                # If no candidate can serve i under the radius, skip (equivalent to 0 <= 1)
                pass
    else:
        for i in I:
            model.addConstr(gp.quicksum(Y[i, j] for j in J) <= 1, name=f"Demand_Service_Constraint_{i}")

    # Constraint: linking — can only serve if the facility is opened
    if allowed_pairs is not None:
        for (i, j) in allowed_pairs:
            model.addConstr(Y[i, j] <= X[j], name=f"Linking_Constraint_{i}_{j}")
    else:
        for i in I:
            for j in J:
                model.addConstr(Y[i, j] <= X[j], name=f"Linking_Constraint_{i}_{j}")

    # Solve
    model.optimize()

    # Results
    selected_sites = []
    if model.status == GRB.OPTIMAL:
        objective_value = model.ObjVal
        for j in J:
            if X[j].X > 0.5:
                selected_sites.append(j)
    else:
        objective_value = None
        print("Model infeasible or not optimal.")

    # Coverage statistics: if allowed_pairs is used, check if a built store can serve i
    total_demand = float(sum(demand_points.values()))
    covered_demand = 0.0
    if allowed_pairs is not None and objective_value is not None:
        built = set(selected_sites)
        from collections import defaultdict
        js_of_i = defaultdict(list)
        for (i, j) in allowed_pairs:
            if j in built:
                js_of_i[i].append(j)
        for i, js in js_of_i.items():
            # If any built j can serve i, mark i as covered
            if len(js) > 0:
                covered_demand += float(demand_points[i])
    coverage_rate = (covered_demand / total_demand) if total_demand > 0 else 0.0

    # Component metrics (weighted as in the objective)
    sum_B_selected = sum(B[j] for j in selected_sites)
    sum_C_selected = sum(C[j] for j in selected_sites)
    sum_rent_selected = sum(candidate_sites[j]['rent'] for j in selected_sites)

    metrics = {
        'market_benefit': parameters['w1'] * covered_demand,
        'agglomeration_benefit': parameters['w3'] * sum_B_selected,
        'competition_cost': parameters['w4'] * sum_C_selected,  # objective subtracts this term
        'rent_cost': parameters['w2'] * sum_rent_selected,      # objective subtracts this term
        'avg_rent': (sum_rent_selected / len(selected_sites)) if len(selected_sites) > 0 else 0.0,
        'sum_rent': sum_rent_selected,
        'sum_B': sum_B_selected,
        'sum_C': sum_C_selected,
    }

    return selected_sites, objective_value, covered_demand, total_demand, coverage_rate, metrics


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

In [ ]:

def render_scale_bar(ax, x, y, segments=2, height=0.01, seg_length=2000, unit='m', linewidth=1.):
    unit_scale_factor = {
        'm': 1,
        'km': 1000,
        'meters': 1,
        'kilometers': 1000,
        'miles': 1609.34,
        'mi': 1609.34,
        'ft': 0.3,
        }
    x_lim = ax.get_xlim()
    y_lim = ax.get_ylim()

    x_per_unit = 1. / (x_lim[1] - x_lim[0])
    y_per_unit = 1. / (y_lim[1] - y_lim[0])

    # base for ticks (0, 1)
    x_base = [x + seg_length * unit_scale_factor[unit] * x_per_unit * i for i in range(0, segments + 1)]
    ax.axhline(y_lim[0] + y / y_per_unit, x_base[0], x_base[-1], c='black')
    y_base = [y, y + height]
    for i in range(segments + 1):
        ax.axvline(x_lim[0] + x_base[i] / x_per_unit, y, y + height, c='black')
        xy = (x_lim[0] + x_base[i] / x_per_unit, y_lim[0] + (y - 0.015) / y_per_unit)  # data_coords
        ax.text(xy[0], xy[1], s='{}'.format(int(seg_length * i)), horizontalalignment='center', verticalalignment='center')
    ax.text(x_lim[0] + (x_base[-1] + 0.02) / x_per_unit, y_lim[0] + (y - 0.015) / y_per_unit,
            s=unit, horizontalalignment='left',
            verticalalignment='center')

In [ ]:

def render_north_arrow(ax, x, y, size, ratio = 1):
    path = [(0, 1), (-ratio, -1), (0, -0.5), (ratio, -1), (0, 1)]
    path = [(i[0] * size + x, i[1] * size + y) for i in path]
    arrow = plt.Polygon(path, color='black', transform=ax.transAxes)
    ax.add_patch(arrow)
    ax.text(x, y-size*2, s = 'N', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

In [ ]:
# === Read HuLaTang data and build dictionaries ===
import os

# Paths
base_dir = "data/HuLaTang"
demand_shp_path = os.path.join(base_dir, "Deman_Point.shp")
candidate_csv_path = os.path.join(base_dir, "Candidate_Point.csv")
existing_csv_path = os.path.join(base_dir, "Hulatang.csv")

# 1) Read demand points (shp). Market demand comes from column All_pop
ls_hlt = gpd.read_file(demand_shp_path)
if not ls_hlt.geom_type.isin(["Point"]).all():
    # If polygons/lines, use centroids
    ls_hlt = ls_hlt.copy()
    ls_hlt["geometry"] = ls_hlt.geometry.centroid
ls_hlt["X"] = ls_hlt.geometry.x
ls_hlt["Y"] = ls_hlt.geometry.y

# Case-insensitive detection of All_pop
all_pop_col = None
for c in ls_hlt.columns:
    if c.lower() == "all_pop":
        all_pop_col = c
        break
if all_pop_col is None:
    raise ValueError("Column 'All_pop' not found in demand shapefile; please check field names.")

# Demand ids: prefer FID/Id, otherwise row index
if "FID" in ls_hlt.columns:
    demand_ids = ls_hlt["FID"].astype(int).tolist()
elif "Id" in ls_hlt.columns:
    demand_ids = ls_hlt["Id"].astype(int).tolist()
else:
    demand_ids = list(range(len(ls_hlt)))

# Build demand_points: {demand_id: All_pop}
demand_points = {demand_ids[i]: float(ls_hlt.iloc[i][all_pop_col]) for i in range(len(ls_hlt))}

# 2) Read candidates (csv): rent from ZUJIN/Zujin; coordinates from point_x_1/point_y_1
encodings = ["utf-8-sig", "gbk", "utf-8"]
for enc in encodings:
    try:
        cand_df = pd.read_csv(candidate_csv_path, encoding=enc)
        break
    except Exception:
        cand_df = None
if cand_df is None:
    raise RuntimeError("Failed to read Candidate_Point.csv; please check file and encoding.")

# Coordinate columns compatibility
x_col = None
y_col = None
for c in cand_df.columns:
    cl = c.lower()
    if cl in ("point_x_1", "x", "point_x"):
        x_col = c if x_col is None else x_col
    if cl in ("point_y_1", "y", "point_y"):
        y_col = c if y_col is None else y_col
if x_col is None or y_col is None:
    raise ValueError("Candidate csv is missing coordinate columns (point_x_1/point_y_1 or X/Y).")

# Rent column compatibility
rent_col = None
for c in cand_df.columns:
    if c.lower() in ("zujin", "租金", "rent"):
        rent_col = c
        break
if rent_col is None:
    raise ValueError("Candidate csv is missing rent column (ZUJIN/租金/rent).")

# Normalize index
cand_df = cand_df.reset_index(drop=True)
J_ids = list(range(len(cand_df)))

# Build candidate_sites: {j: {coords: (x,y), rent: F_j}}
candidate_sites = {
    j: {
        "coords": (float(cand_df.loc[j, x_col]), float(cand_df.loc[j, y_col])),
        "rent": float(cand_df.loc[j, rent_col])
    }
    for j in J_ids
}

# 3) Read existing stores (csv), with Brand, Ak, Pk and coordinates X, Y
for enc in encodings:
    try:
        stores_df = pd.read_csv(existing_csv_path, encoding=enc)
        break
    except Exception:
        stores_df = None
if stores_df is None:
    raise RuntimeError("Failed to read Hulatang.csv; please check file and encoding.")

# Column compatibility
store_x_col = None
store_y_col = None
for c in stores_df.columns:
    if c.lower() == "x":
        store_x_col = c
    if c.lower() == "y":
        store_y_col = c
if store_x_col is None or store_y_col is None:
    # Fallback: some tables use POINT_X/POINT_Y
    for c in stores_df.columns:
        if c.lower() in ("point_x", "pointx"):
            store_x_col = c
        if c.lower() in ("point_y", "pointy"):
            store_y_col = c
if store_x_col is None or store_y_col is None:
    raise ValueError("Store csv is missing coordinate columns (X/Y or POINT_X/POINT_Y).")

Ak_col = None
Pk_col = None
for c in stores_df.columns:
    if c.lower() == "ak":
        Ak_col = c
    if c.lower() == "pk":
        Pk_col = c
if Ak_col is None or Pk_col is None:
    raise ValueError("Store csv is missing Ak or Pk columns.")

stores_df = stores_df.reset_index(drop=True)
K_ids = list(range(len(stores_df)))

# Build existing_stores: {k: {coords: (x,y), Ak: , Pk: , Brand: }}
existing_stores = {
    k: {
        "coords": (float(stores_df.loc[k, store_x_col]), float(stores_df.loc[k, store_y_col])),
        "Ak": float(stores_df.loc[k, Ak_col]),
        "Pk": float(stores_df.loc[k, Pk_col]),
        "Brand": str(stores_df.loc[k, "Brand"]) if "Brand" in stores_df.columns else ""
    }
    for k in K_ids
}

print(f"Demand: {len(demand_points)}, Candidates: {len(candidate_sites)}, Existing stores: {len(existing_stores)}")


In [ ]:
# === Coordinate normalization (unified for candidates and existing stores) ===
# Extract raw coordinates from candidate_sites / existing_stores
cand_xy = np.array([candidate_sites[j]["coords"] for j in sorted(candidate_sites.keys())], dtype=float)
store_xy = np.array([existing_stores[k]["coords"] for k in sorted(existing_stores.keys())], dtype=float)
all_xy = np.vstack([cand_xy, store_xy])

# Compute scale S and normalize to [0, 1]
x = all_xy[:, 0]
y = all_xy[:, 1]
min_x, max_x = x.min(), x.max()
min_y, max_y = y.min(), y.max()
S = max(max_x - min_x, max_y - min_y)
Nx = (x - min_x) / S
Ny = (y - min_y) / S
norm_xy = np.vstack([Nx, Ny]).T

# Backfill norm_coords for candidate_sites / existing_stores
J_sorted = sorted(candidate_sites.keys())
K_sorted = sorted(existing_stores.keys())
for idx, j in enumerate(J_sorted):
    candidate_sites[j]["norm_coords"] = (float(norm_xy[idx, 0]), float(norm_xy[idx, 1]))
for idx, k in enumerate(K_sorted):
    existing_stores[k]["norm_coords"] = (float(norm_xy[len(J_sorted)+idx, 0]), float(norm_xy[len(J_sorted)+idx, 1]))

print(f"Normalization done. Scale S = {S:.3f}")


In [ ]:
# === Distance matrix (using normalized coordinates) ===
import numpy as np

J_sorted = sorted(candidate_sites.keys())
K_sorted = sorted(existing_stores.keys())
cand_norm = np.array([candidate_sites[j]["norm_coords"] for j in J_sorted], dtype=float)
store_norm = np.array([existing_stores[k]["norm_coords"] for k in K_sorted], dtype=float)
cd = cand_norm[:, None, :] - store_norm[None, :, :]
dists_norm = np.sqrt((cd ** 2).sum(axis=2))

distance_matrix = {}
for jj, j in enumerate(J_sorted):
    for kk, k in enumerate(K_sorted):
        distance_matrix[(j, k)] = float(dists_norm[jj, kk])

print(f"Distance matrix size (normalized): {len(J_sorted)} x {len(K_sorted)} built")


In [ ]:
# === Demand normalization and service-radius filtering (R=1000m) ===
# 1) Original demand coordinates
demand_xy = ls_hlt[["X", "Y"]].to_numpy(float)

# 2) Normalize using the same min/max and S as candidates/stores (joint range)
cand_xy = np.array([candidate_sites[j]["coords"] for j in sorted(candidate_sites.keys())], dtype=float)
store_xy = np.array([existing_stores[k]["coords"] for k in sorted(existing_stores.keys())], dtype=float)
all_xy_cs = np.vstack([cand_xy, store_xy])
min_x_cs, max_x_cs = all_xy_cs[:, 0].min(), all_xy_cs[:, 0].max()
min_y_cs, max_y_cs = all_xy_cs[:, 1].min(), all_xy_cs[:, 1].max()
S = max(max_x_cs - min_x_cs, max_y_cs - min_y_cs)  # same S as normalization step

Nx_d = (demand_xy[:, 0] - min_x_cs) / S
Ny_d = (demand_xy[:, 1] - min_y_cs) / S
demand_norm = np.vstack([Nx_d, Ny_d]).T

# 3) Radius filter
R_m = 1000.0
R_norm = R_m / S
J_sorted = sorted(candidate_sites.keys())

cand_norm = np.array([candidate_sites[j]['norm_coords'] for j in J_sorted], dtype=float)

# Demand id mapping (consistent with demand_points keys)
if "FID" in ls_hlt.columns:
    id_series = ls_hlt["FID"].astype(int)
elif "Id" in ls_hlt.columns:
    id_series = ls_hlt["Id"].astype(int)
else:
    id_series = pd.Series(range(len(ls_hlt)))

allowed_pairs = []
for j_idx, j in enumerate(J_sorted):
    diff = demand_norm - cand_norm[j_idx]
    d = np.sqrt((diff ** 2).sum(axis=1))
    for ridx in np.where(d <= R_norm)[0]:
        i = int(id_series.iloc[ridx])
        allowed_pairs.append((i, j))

allowed_pairs = set(allowed_pairs)
print(f"Service radius R = {R_m} m (normalized {R_norm:.8f}), allowed pairs: {len(allowed_pairs)}")


In [ ]:
# === Run the model (δ uses normalized scale) ===
params = {
    'w1': 0.4,
    'w3': 0.1,
    'w2': 0.25,
    'w4': 0.25,
    'delta_agg': 700.0 / S,
    'delta_comp': 150.0 / S,
}

p_new = 20

selected_sites, objective_value, covered_demand, total_demand, coverage_rate, metrics = solve_hulatang_location_model(
    demand_points=demand_points,
    candidate_sites=candidate_sites,
    existing_stores=existing_stores,
    p=p_new,
    parameters=params,
    distance_matrix=distance_matrix,
    allowed_pairs=allowed_pairs
)
print(f"Optimal objective: {objective_value:.0f}")
print(f"Market coverage benefit: {metrics['market_benefit']:.0f}")
print(f"Agglomeration benefit: {metrics['agglomeration_benefit']:.0f}")
print(f"Competition cost: {-metrics['competition_cost']:.0f}")  # subtracted in objective
print(f"Rent cost: {-metrics['rent_cost']:.0f}")                 # subtracted in objective
print(f"Average rent per store: {metrics['avg_rent']:.0f}")
print(f"Covered demand: {covered_demand:.0f} / Total demand: {total_demand:.0f}")
print(f"Coverage rate: {coverage_rate:.2%}")

# Inspect selected sites
if len(selected_sites) > 0:
    sel_df = cand_df.loc[selected_sites, [x_col, y_col, rent_col]].copy()
    sel_df["candidate_id"] = selected_sites
    display(sel_df.head())


In [ ]:
from matplotlib import colors

fig, ax = plt.subplots(figsize=(20, 18))

try:
    cmap = plt.cm.Blues
    new_cmap = colors.ListedColormap(cmap(np.linspace(0.15, 1, 256)))
    ls_hlt.plot(ax=ax, column=ls_hlt[all_pop_col], k=5, markersize=15, cmap=new_cmap, label='Market (All_pop)')
except Exception:
    pass

if len(selected_sites) > 0:
    opt_sites = cand_df.loc[selected_sites]
    ax.scatter(opt_sites[x_col], opt_sites[y_col], c='C1', marker='+', s=100, label='Selected Sites')

ax.axis('scaled')
ax.tick_params(axis='both', left=False, top=False, right=False,
               bottom=False, labelleft=False, labeltop=False,
               labelright=False, labelbottom=False)
ax.set_title("Optimized Site Selection (Normalized distances)", fontsize=20)
render_scale_bar(ax=ax, x=0.05, y=0.05)
render_north_arrow(ax=ax, x=0.95, y=0.95, size=0.01, ratio=0.7)
ax.legend(loc='lower right', markerscale=10)


In [ ]:
import os
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib.lines import Line2D
from matplotlib import patheffects as pe

def plot_result_pretty(ls, opt_sites, radius_m=2000, roads_path="data/HuLaTang/Road_Network.shp",
                       demand_gridsize=60, demand_cmap="magma", roads_color="#9e9e9e", roads_alpha=0.35,
                       norm_mode="log", vmin_q=0.10, vmax_q=0.995, gamma=0.6, reduce="sum",
                       overlay_points=True, overlay_points_size=6, overlay_points_alpha=0.28,
                       overlay_hex_grid=False, hex_grid_color="#222222", hex_grid_alpha=0.18, hex_grid_lw=0.25,
                       circle_style="ring", show_heat=True, show_colorbar=False, lang="zh", title=None):
    """
    Publication-quality plotting for site selection: demand heat (hexbin) + roads + facilities + service circles.

    Args:
        ls (GeoDataFrame): demand points containing POINT_X/POINT_Y or X/Y and optionally All_pop.
        opt_sites (DataFrame): selected rows with x/y columns (point_x_1/POINT_X...).
        radius_m (float): service radius (same units as coordinates).
        roads_path (str): path to road shapefile.
        demand_gridsize (int): hexbin grid size.
        demand_cmap (str): colormap for demand heat.
        roads_color (str): road color.
        roads_alpha (float): road alpha.
        norm_mode (str): 'log' | 'power' | 'linear'.
        vmin_q, vmax_q (float): demand quantile clipping, for display only.
        gamma (float): gamma for PowerNorm.
        reduce (str): 'sum' or 'mean' for hexbin aggregation.
        overlay_points (bool): overlay demand points.
        circle_style (str): 'fill' | 'ring' service circle style.
        show_colorbar (bool): whether to display colorbar.
        lang (str): 'zh' or 'en' for labels.
        title (str|None): custom title.
    """
    L = {
        'zh': {
            'road': '道路网络', 'demand': '需求点', 'selected': '新建门店', 'current': '现有门店',
            'title': '选址结果与需求强度', 'subtitle': lambda n, r: f"门店数：{n}  半径：{r} m"
        },
        'en': {
            'road': 'Road network', 'demand': 'Demand points', 'selected': 'Selected facilities', 'current': 'Current sites',
            'title': 'Optimized Facilities with Demand Heat', 'subtitle': lambda n, r: f"Facilities: {n}  Radius: {r} m"
        }
    }
    LL = L.get(lang, L['zh'])

    pop_col = None
    if 'speed_pct_freeflow_rev' in ls.columns:
        pop_col = 'speed_pct_freeflow_rev'
    else:
        for c in ls.columns:
            if str(c).lower() == 'all_pop':
                pop_col = c
                break
    if 'X' in ls.columns and 'Y' in ls.columns:
        dx, dy = 'X', 'Y'
    else:
        dx, dy = 'POINT_X', 'POINT_Y'
    if dx not in ls.columns or dy not in ls.columns:
        raise ValueError("ls must contain POINT_X/POINT_Y or X/Y columns.")

    def pick_xy_cols(df):
        x_col = None
        y_col = None
        for c in df.columns:
            cl = str(c).lower()
            if cl in ('point_x_1', 'x', 'point_x', 'pointx') and x_col is None:
                x_col = c
            if cl in ('point_y_1', 'y', 'point_y', 'pointy') and y_col is None:
                y_col = c
        if x_col is None and 'POINT_X' in df.columns: x_col = 'POINT_X'
        if y_col is None and 'POINT_Y' in df.columns: y_col = 'POINT_Y'
        if x_col is None or y_col is None:
            raise ValueError('opt_sites missing XY columns (point_x_1/point_y_1 or X/Y or POINT_X/POINT_Y).')
        return x_col, y_col

    x_col, y_col = pick_xy_cols(opt_sites)

    fig, ax = plt.subplots(figsize=(20, 16))

    if isinstance(roads_path, str) and os.path.exists(roads_path):
        try:
            roads = gpd.read_file(roads_path)
            try:
                roads.plot(ax=ax, color=roads_color, linewidth=0.4, alpha=roads_alpha, zorder=1, label=LL['road'])
            except Exception:
                pass
        except Exception:
            pass

    try:
        if show_heat:
            values = ls[pop_col].to_numpy() if pop_col is not None else None
            if values is not None:
                vmin = float(np.quantile(values, vmin_q)) if 0 <= vmin_q < 1 else None
                vmax = float(np.quantile(values, vmax_q)) if 0 < vmax_q <= 1 else None
                if vmin is not None and vmax is not None and vmax > vmin:
                    values_clipped = np.clip(values, vmin, vmax)
                else:
                    values_clipped = values
                if norm_mode == 'log':
                    from matplotlib.colors import LogNorm
                    norm = LogNorm(vmin=max(values_clipped.min(), 1e-6), vmax=values_clipped.max())
                elif norm_mode == 'power':
                    from matplotlib.colors import PowerNorm
                    norm = PowerNorm(gamma=gamma, vmin=values_clipped.min(), vmax=values_clipped.max())
                else:
                    norm = None
            else:
                values_clipped = None
                norm = None

            reducer = np.sum if reduce == 'sum' else np.mean
            hb = ax.hexbin(ls[dx].to_numpy(), ls[dy].to_numpy(),
                           C=values_clipped,
                           reduce_C_function=reducer if values_clipped is not None else None,
                           gridsize=demand_gridsize, cmap=demand_cmap, bins=None, mincnt=1,
                           linewidths=0, alpha=0.92, zorder=5, norm=norm)
            try:
                hb.set_edgecolor('face')
            except Exception:
                pass
            if show_colorbar and values is not None:
                cbar = fig.colorbar(hb, ax=ax, shrink=0.8)
                cbar.ax.tick_params(labelsize=10)
    except Exception:
        if show_heat:
            sc = ax.scatter(ls[dx], ls[dy], c=ls[pop_col] if pop_col is not None else '#9ecae1',
                            s=8, cmap=demand_cmap, edgecolors='none', zorder=5)
            if show_colorbar and pop_col is not None:
                fig.colorbar(sc, ax=ax, shrink=0.8)

    if overlay_points:
        try:
            ax.scatter(ls[dx], ls[dy], s=overlay_points_size, c='#2c3e50', alpha=overlay_points_alpha,
                       linewidths=0, zorder=6, label=LL['demand'])
        except Exception:
            pass

    legend_flag = {'selected': False, 'current': False}
    for _, row in opt_sites.iterrows():
        cx = float(row[x_col])
        cy = float(row[y_col])
        is_current = ('current' in opt_sites.columns and bool(row['current']) is True)
        if is_current:
            coll = ax.scatter(cx, cy, s=46, marker='o', facecolor='white', edgecolor='red', linewidths=1.2,
                              zorder=10, label=LL['current'] if not legend_flag['current'] else None)
            try:
                coll.set_path_effects([pe.withStroke(linewidth=2.2, foreground='white')])
            except Exception:
                pass
            legend_flag['current'] = True
            circ_kwargs = dict(facecolor='none', edgecolor='red', lw=1.0, alpha=0.9, zorder=9)
            if circle_style == 'fill':
                circ_kwargs.update(facecolor='none')
            ax.add_artist(plt.Circle((cx, cy), radius_m, **circ_kwargs))
        else:
            coll = ax.scatter(cx, cy, s=52, marker='o', facecolor='#FF8C42', edgecolor='white', linewidths=0.8,
                              zorder=11, label=LL['selected'] if not legend_flag['selected'] else None)
            try:
                coll.set_path_effects([pe.withStroke(linewidth=2.0, foreground='white')])
            except Exception:
                pass
            legend_flag['selected'] = True
            if circle_style == 'fill':
                circ = plt.Circle((cx, cy), radius_m, facecolor='#FF8C42', edgecolor='#FF8C42', lw=0.6, alpha=0.12, zorder=8)
            else:
                circ = plt.Circle((cx, cy), radius_m, facecolor='none', edgecolor='#FF8C42', lw=1.2, ls='--', alpha=0.85, zorder=9)
            ax.add_artist(circ)

    ax.axis('scaled')
    ax.tick_params(axis='both', left=False, top=False, right=False,
                   bottom=False, labelleft=False, labeltop=False,
                   labelright=False, labelbottom=False)
    ax.grid(False)
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_linewidth(1.0)
        spine.set_edgecolor('#333333')
    ax.margins(x=0.02, y=0.02)

    ttl = title if title is not None else LL['title']
    ax.set_title(ttl + "\n" + LL['subtitle'](len(opt_sites), int(radius_m)), fontsize=18, pad=12)
    try:
        render_scale_bar(ax=ax, x=0.05, y=0.05)
        render_north_arrow(ax=ax, x=0.95, y=0.95, size=0.012, ratio=0.7)
    except Exception:
        pass

    legend_marker_size = 8
    has_current = ('current' in opt_sites.columns and bool(np.any(opt_sites['current'].astype(bool))))
    custom_handles = []
    # road
    custom_handles.append(Line2D([0], [0], color=roads_color, lw=1.0, alpha=0.6, label=LL['road']))
    # current (optional)
    if has_current:
        custom_handles.append(Line2D([0], [0], marker='o', color='none', markerfacecolor='white',
                                     markeredgecolor='red', markeredgewidth=1.2, markersize=legend_marker_size,
                                     label=LL['current']))
    # demand
    if overlay_points:
        custom_handles.append(Line2D([0], [0], marker='o', color='none', markerfacecolor='#2c3e50',
                                     markersize=legend_marker_size, alpha=overlay_points_alpha, label=LL['demand']))
    # selected
    if len(opt_sites) > 0:
        custom_handles.append(Line2D([0], [0], marker='o', color='none', markerfacecolor='#FF8C42',
                                     markeredgecolor='white', markeredgewidth=0.8, markersize=legend_marker_size,
                                     label=LL['selected']))
    ax.legend(handles=custom_handles, loc='lower right', markerscale=1.0, frameon=True, framealpha=0.85, fontsize=11)
    fig.tight_layout()
    return ax


In [ ]:
# Demo call for pretty plot (Gurobi)
try:
    opt_sites = cand_df.loc[selected_sites]
except Exception:
    opt_sites = cand_df.iloc[:30]

ax = plot_result_pretty(
    ls_hlt,
    opt_sites,
    radius_m=1000,
    roads_path="data/HuLaTang/Road_Network.shp",
    demand_gridsize=80,
    demand_cmap="inferno",
    norm_mode="log",
    vmin_q=0.12,
    vmax_q=0.995,
    reduce="sum",
    circle_style="ring",
    show_heat=False,
    lang="en",
    title=None
)
# 16:9 canvas (for PPT) and save HD image
fig = ax.figure
fig.set_size_inches(20, 11.25)
fig.tight_layout()
fig.savefig("hulatang_result_pretty_SA.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
# Print and save selected_sites (JSON only)
import os, json

try:
    print("Gurobi selected site indices (selected_sites):\n", selected_sites)
except NameError:
    raise RuntimeError("'selected_sites' is undefined. Please run the solving cell first.")

# Save to results directory (JSON only)
out_dir = "results"
os.makedirs(out_dir, exist_ok=True)
json_path = os.path.join(out_dir, "hulatang_gurobi_selected_sites.json")
with open(json_path, "w", encoding="utf-8") as f:
    json.dump({"selected_sites": selected_sites}, f, ensure_ascii=False, indent=2)
print("Saved JSON:", json_path)
